In [10]:
import pandas as pd
from Levenshtein import distance

In [49]:
bulba_df = pd.read_json('scrap/bulba4.json')
evol_df  = pd.read_csv('pokemon_species.csv')

In [50]:
bulba_df

,name,nat_idx,image_link,image_path
0,Bulbasaur,#0001,https://archives.bulbagarden.net/media/upload/...,images/#0001_Bulbasaur.png
1,Raichu,#0026,https://archives.bulbagarden.net/media/upload/...,images/#0026_Raichu.png
2,Weedle,#0013,https://archives.bulbagarden.net/media/upload/...,images/#0013_Weedle.png
3,Pikachu,#0025,https://archives.bulbagarden.net/media/upload/...,images/#0025_Pikachu.png
4,Kakuna,#0014,https://archives.bulbagarden.net/media/upload/...,images/#0014_Kakuna.png
...,...,...,...,...
1020,Wartortle,#0008,https://archives.bulbagarden.net/media/upload/...,images/#0008_Wartortle.png
1021,Charmander,#0004,https://archives.bulbagarden.net/media/upload/...,images/#0004_Charmander.png
1022,Charmeleon,#0005,https://archives.bulbagarden.net/media/upload/...,images/#0005_Charmeleon.png
1023,Venusaur,#0003,https://archives.bulbagarden.net/media/upload/...,images/#0003_Venusaur.png


In [51]:
evol_df.head()

,id,identifier,generation_id,evolves_from_species_id,evolution_chain_id,color_id,shape_id,habitat_id,gender_rate,capture_rate,base_happiness,is_baby,hatch_counter,has_gender_differences,growth_rate_id,forms_switchable,is_legendary,is_mythical,order,conquest_order
0,1,bulbasaur,1,NaN,1,5,8,3.0,1,45,50.0,0,20.0,0,4,0,0,0,1,NaN
1,2,ivysaur,1,1.0,1,5,8,3.0,1,45,50.0,0,20.0,0,4,0,0,0,2,NaN
2,3,venusaur,1,2.0,1,5,8,3.0,1,45,50.0,0,20.0,1,4,1,0,0,3,NaN
3,4,charmander,1,NaN,2,8,6,4.0,1,45,50.0,0,20.0,0,4,0,0,0,4,109.0
4,5,charmeleon,1,4.0,2,8,6,4.0,1,45,50.0,0,20.0,0,4,0,0,0,5,110.0


In [52]:
from tqdm import tqdm
bulba_to_evol = {}
for i, row_bulba in tqdm(bulba_df.iterrows()):
    min_dist = 1000
    min_name = ''
    for j, row_evol in evol_df.iterrows():
        dist = distance(row_bulba['name'].lower(), row_evol['identifier'].lower())
        if dist < min_dist:
            min_dist = dist
            min_name = row_evol['identifier']
            
    bulba_to_evol[row_bulba['name']] = min_name
    
    if "Charmeleon" in row_bulba['name']:
        print(row_bulba['name'], min_name, min_dist)

1025it [00:37, 27.06it/s]

Charmeleon charmeleon 0


In [56]:
bulba_to_evol["Nidoran♂"] = "nidoran-m"

In [57]:
def find_duplicated_names(liste):
    seen = set()
    duplicates = set()
    for name in liste:
        if name in seen:
            duplicates.add(name)
        seen.add(name)
    return duplicates
find_duplicated_names(bulba_to_evol.values())

set()

In [59]:
import json
with open('bulba_to_evol.json', 'w') as f:
    json.dump(bulba_to_evol, f, indent=4, ensure_ascii=False)

In [60]:
evol_to_bulba = {v: k for k, v in bulba_to_evol.items()}

# make_dataset

make a list of tuple of (origin, evolution) from the given dataset

In [61]:
dataset = []

bulba_item = "Dodrio"

def find_evolution(bulba_item):
    evol_item = evol_df[evol_df['identifier'] == bulba_to_evol[bulba_item]]
    evolution_id = evol_item['evolves_from_species_id'].values[0]
    if evolution_id is None or 'nan' in str(evolution_id):
        return None
    evolution_name = evol_df[evol_df['id'] == evolution_id]['identifier'].values[0]
    bulba_evolution_name = evol_to_bulba[evolution_name]
    return bulba_evolution_name

for bulba_item in bulba_to_evol.keys():
    try:
        evolution = find_evolution(bulba_item)
        if evolution is not None:
            dataset.append((evolution, bulba_item))
    except Exception as e:
        print(bulba_item)
        print(e)
        continue
    

In [67]:
# make metadata.csv file
def find_image_for_name(name):
    for i, row in bulba_df.iterrows():
        if row['name'] == name:
            return row["image_path"]
    raise Exception("Image not found for name: " + name)

In [68]:
dataset_image = [(find_image_for_name(evolution), find_image_for_name(bulba)) for evolution, bulba in dataset]

In [88]:
# convert to csv
dataset_df = pd.DataFrame(dataset_image, columns=["origine", "evolution"])
dataset_df.to_csv(
    "/workspaces/pokemon-infinite-evolution/scrap/dataset/images/metadata.csv",
    index=False,
)

In [91]:
from datasets import load_dataset

hf_dataset = load_dataset("/workspaces/pokemon-infinite-evolution/scrap/dataset")

Generating train split: 484 examples [00:00, 24779.59 examples/s]

images/#0025_Pikachu.png images/#0026_Raichu.png
images/#0172_Pichu.png images/#0025_Pikachu.png
images/#0013_Weedle.png images/#0014_Kakuna.png
images/#0023_Ekans.png images/#0024_Arbok.png
images/#0021_Spearow.png images/#0022_Fearow.png
images/#0014_Kakuna.png images/#0015_Beedrill.png
images/#0016_Pidgey.png images/#0017_Pidgeotto.png
images/#0017_Pidgeotto.png images/#0018_Pidgeot.png
images/#0011_Metapod.png images/#0012_Butterfree.png
images/#0714_Noibat.png images/#0715_Noivern.png
images/#0627_Rufflet.png images/#0628_Braviary.png
images/#0884_Duraludon.png images/#1018_Archaludon.png
images/#1011_Dipplin.png images/#1019_Hydrapple.png
images/#0840_Applin.png images/#1011_Dipplin.png
images/#1012_Poltchageist.png images/#1013_Sinistcha.png
images/#0997_Arctibax.png images/#0998_Baxcalibur.png
images/#0999_Gimmighoul.png images/#1000_Gholdengo.png
images/#0996_Frigibax.png images/#0997_Arctibax.png
images/#0206_Dunsparce.png images/#0982_Dudunsparce.png
images/#0625_Bisharp.png

In [95]:
hf_dataset.push_to_hub("PhilSad/pokemon_evolution", private=True)

Uploading the dataset shards: 100%|██████████| 2/2 [00:21<00:00, 10.88s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/PhilSad/pokemon_evolution/commit/0ea047196100e1d44cb62eb9890a525c5dc45d5c', commit_message='Upload dataset', commit_description='', oid='0ea047196100e1d44cb62eb9890a525c5dc45d5c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/PhilSad/pokemon_evolution', endpoint='https://huggingface.co', repo_type='dataset', repo_id='PhilSad/pokemon_evolution'), pr_revision=None, pr_num=None)

In [96]:
hf_dataset["train"]

Dataset({
    features: ['origin', 'evolution'],
    num_rows: 484
})